# ⚽ EPL Moneyball AI: Predicting Match Outcomes with XGBoost
**Author:** Peer Nagar
**Accuracy:** 54.08% (Validated on Test Set)

### 🚀 Project Overview
This project utilizes historical Premier League data to predict match results and identify **Value Bets**.
It leverages **XGBoost** with optimized hyperparameters and advanced feature engineering, including:
* **Team Form & Momentum:** Rolling averages of recent performance.
* **Interaction Features:** Direct comparison between Home Attack vs. Away Defense.
* **Time Decay:** Giving double weight to recent matches (2024-2025).

### 🛠️ Methodology
1.  **Data Loading:** Aggregating 5 seasons of match data.
2.  **Feature Engineering:** Creating dynamic time-series features.
3.  **Model Training:** Using pre-optimized hyperparameters found via RandomizedSearchCV.
4.  **Deployment:** Generating a real-time betting report for the upcoming round.

In [1]:
import pandas as pd
import requests
import json
import io
from datetime import datetime
import numpy as np
from google.colab import drive
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error

# --- 1. Define Seasons to Download ---
# We want the last 5 seasons dynamically
current_year = datetime.now().year
# If we are in Jan-July, the current season started last year. If Aug-Dec, it started this year.
start_year = current_year if datetime.now().month > 7 else current_year - 1

seasons = []
for i in range(5): # Go back 5 seasons
    y_start = start_year - i
    y_end = y_start + 1
    # The website format is 4 digits (e.g., 2324 for the 2023/2024 season)
    season_str = f"{str(y_start)[-2:]}{str(y_end)[-2:]}"
    seasons.append(season_str)

# Chronological order (oldest to newest)
seasons = sorted(seasons)

# --- 2. Function to Download Data from the Web ---
base_url = "https://www.football-data.co.uk/mmz4281/"
data_frames = []

print(f"🔄 Connecting to football-data.co.uk...")

for season in seasons:
    url = f"{base_url}{season}/E0.csv"
    try:
        s = requests.get(url).content
        df_temp = pd.read_csv(io.StringIO(s.decode('utf-8')))
        df_temp['Season_File'] = season # Marker to identify the season source
        data_frames.append(df_temp)

        # --- התיקון כאן: עיצוב המחרוזת רק לתצוגה ---
        season_display = f"{season[:2]}/{season[2:]}"
        print(f"   ✅ Downloaded season {season_display}")

    except Exception as e:
        # גם כאן נתקן לתצוגה יפה
        season_display = f"{season[:2]}/{season[2:]}"
        print(f"   ⚠️ Could not download season {season_display}: {e}")

# --- 3. Merge and Clean ---
if data_frames:
    df = pd.concat(data_frames, ignore_index=True)

    # Date conversion (The site uses British format DD/MM/YYYY)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.sort_values(by='Date').reset_index(drop=True)

    # Filter relevant columns
    cols_to_keep = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
                    'HS', 'AS', 'HST', 'AST', 'B365H', 'B365D', 'B365A']
    # Keep only columns that exist (sometimes column names change slightly between seasons)
    existing = [c for c in cols_to_keep if c in df.columns]
    df = df[existing].copy()

    # Drop empty rows (matches not yet played or missing results in the main file)
    df.dropna(subset=['FTR'], inplace=True)

    print(f"\n🎉 SUCCESS: Loaded {len(df)} matches directly from the web.")
    print(f"   Data Range: {df['Date'].min().date()} to {df['Date'].max().date()}")
else:
    print("❌ Error: No data loaded.")

🔄 Connecting to football-data.co.uk...
   ✅ Downloaded season 21/22
   ✅ Downloaded season 22/23
   ✅ Downloaded season 23/24
   ✅ Downloaded season 24/25
   ✅ Downloaded season 25/26

🎉 SUCCESS: Loaded 1720 matches directly from the web.
   Data Range: 2021-08-13 to 2026-01-04


## 🧹 Data Cleaning & Preprocessing
We filter the raw dataset to keep only the essential columns:
* **Match Info:** Date, Teams, Goals (FTHG, FTAG).
* **Stats:** Shots, Corners, Fouls (used for deeper analysis if needed).
* **Odds:** Bet365 odds (Home, Draw, Away) to calculate implied probabilities.

In [2]:
# Select only relevant columns for analysis and modeling
cols_to_keep = [
    'Date', 'HomeTeam', 'AwayTeam',
    'FTHG', 'FTAG', 'FTR',           # Goals and Results
    'HS', 'AS', 'HST', 'AST',        # Shots stats
    'B365H', 'B365D', 'B365A',       # Betting Odds
    'Season_File'                    # Helper column
]

# Keep only existing columns (handling potential missing columns in older files)
existing_cols = [c for c in cols_to_keep if c in df.columns]
df = df[existing_cols].copy()

# Drop rows with missing critical data (Results or Odds)
df.dropna(subset=['FTR', 'B365H'], inplace=True)

print(f"Clean Data Shape: {df.shape}")
display(df.tail())

Clean Data Shape: (1720, 13)


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,B365H,B365D,B365A
1715,2026-01-04,Newcastle,Crystal Palace,2,0,H,12,11,7,1,1.70,3.90,4.75
1716,2026-01-04,Fulham,Liverpool,2,2,D,8,10,2,2,3.80,3.75,1.91
1717,2026-01-04,Leeds,Man United,1,1,D,11,15,3,2,2.70,3.30,2.63
1718,2026-01-04,Everton,Brentford,2,4,A,14,11,6,7,2.35,3.25,3.10
1719,2026-01-04,Man City,Chelsea,1,1,D,14,8,3,3,1.60,4.75,4.50


## ⚙️ Advanced Feature Engineering
This is the core of the project. Raw stats (like "Shots on Target") are post-match metrics. To predict the future, we need **historical context**.

We construct the following features for every match, based on the **past 5 games**:
1.  **Form:** Rolling average of points earned.
2.  **Attacking Strength:** Average goals scored.
3.  **Defensive Weakness:** Average goals conceded.
4.  **Momentum:** Recent streak (last 3 games).
5.  **Home/Away Factor:** How well the team performs specifically at home vs. away.



In [3]:
def process_advanced_features(input_df):
    """
    Transforms match-by-match data into team-centric features with rolling averages.
    """
    # Define base columns needed for calculation
    base_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']

    # Create Home Stats DataFrame
    home_stats = input_df[base_cols].copy()
    home_stats['Team'] = home_stats['HomeTeam']
    home_stats['IsHome'] = 1
    home_stats['GoalsScored'] = home_stats['FTHG']
    home_stats['GoalsConceded'] = home_stats['FTAG']
    home_stats['Points'] = home_stats['FTR'].apply(lambda x: 3 if x == 'H' else (1 if x == 'D' else 0))

    # Create Away Stats DataFrame
    away_stats = input_df[base_cols].copy()
    away_stats['Team'] = away_stats['AwayTeam']
    away_stats['IsHome'] = 0
    away_stats['GoalsScored'] = away_stats['FTAG']
    away_stats['GoalsConceded'] = away_stats['FTHG']
    away_stats['Points'] = away_stats['FTR'].apply(lambda x: 3 if x == 'A' else (1 if x == 'D' else 0))

    # Combine and Sort
    team_stats = pd.concat([home_stats, away_stats]).sort_values(['Team', 'Date'])

    # --- Rolling Calculations ---

    # 1. General Form (Last 5 games)
    team_stats['Form_L5'] = team_stats.groupby('Team')['Points'].transform(lambda x: x.rolling(5).mean().shift())

    # 2. Attack & Defense (Last 5 games)
    team_stats['Attack_L5'] = team_stats.groupby('Team')['GoalsScored'].transform(lambda x: x.rolling(5).mean().shift())
    team_stats['Defense_L5'] = team_stats.groupby('Team')['GoalsConceded'].transform(lambda x: x.rolling(5).mean().shift())

    # 3. Momentum (Sum of points in last 3 games)
    team_stats['Momentum_L3'] = team_stats.groupby('Team')['Points'].transform(lambda x: x.rolling(3).sum().shift())

    # 4. Specific Home/Away Factor
    team_stats['Home_Factor'] = team_stats[team_stats['IsHome']==1].groupby('Team')['Points'].transform(lambda x: x.rolling(5).mean().shift())
    team_stats['Away_Factor'] = team_stats[team_stats['IsHome']==0].groupby('Team')['Points'].transform(lambda x: x.rolling(5).mean().shift())

    # --- Merge back to Match Data ---
    cols_to_merge = ['Date', 'Team', 'Form_L5', 'Attack_L5', 'Defense_L5', 'Momentum_L3', 'Home_Factor', 'Away_Factor']

    df_merged = input_df.copy()

    # Merge Home Features
    df_merged = df_merged.merge(team_stats[cols_to_merge], left_on=['Date', 'HomeTeam'], right_on=['Date', 'Team'], how='left')
    df_merged.rename(columns={
        'Form_L5': 'Home_Form', 'Attack_L5': 'Home_Attack', 'Defense_L5': 'Home_Defense',
        'Momentum_L3': 'Home_Momentum', 'Home_Factor': 'Home_HomeFactor'
    }, inplace=True)
    df_merged.drop(columns=['Team', 'Away_Factor'], inplace=True)

    # Merge Away Features
    df_merged = df_merged.merge(team_stats[cols_to_merge], left_on=['Date', 'AwayTeam'], right_on=['Date', 'Team'], how='left')
    df_merged.rename(columns={
        'Form_L5': 'Away_Form', 'Attack_L5': 'Away_Attack', 'Defense_L5': 'Away_Defense',
        'Momentum_L3': 'Away_Momentum', 'Away_Factor': 'Away_AwayFactor'
    }, inplace=True)
    df_merged.drop(columns=['Team', 'Home_Factor'], inplace=True)

    # Clean initial rows with NaNs
    df_merged.dropna(inplace=True)
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

    # --- NEW: Add Interaction Features (The Accuracy Boosters) ---
    df_merged['Diff_Form'] = df_merged['Home_Form'] - df_merged['Away_Form']
    df_merged['Diff_Attack_Defense'] = df_merged['Home_Attack'] - df_merged['Away_Defense']
    df_merged['Diff_Momentum'] = df_merged['Home_Momentum'] - df_merged['Away_Momentum']

    return df_merged

# Apply the function
df_advanced = process_advanced_features(df)
print(f"Engineered Data Shape (Optimized): {df_advanced.shape}")

Engineered Data Shape (Optimized): (1553, 26)


## 🤖 Model Training (XGBoost)
We train two separate models:
1.  **Winner Classifier:** Predicts Home Win / Draw / Away Win.
2.  **Goals Regressor:** Predicts the total number of goals (for Over/Under markets).

**Key Technique:** We use **Time Decay weighting**. Games played after August 2024 get double the weight (`2.0`) compared to older games. This helps the model adapt to the most recent team rosters and managerial changes.

In [4]:
from sklearn.model_selection import RandomizedSearchCV

# 1. Define Optimized Feature List
features = [
    'Home_Form', 'Away_Form',
    'Home_Attack', 'Away_Attack',
    'Home_Defense', 'Away_Defense',
    'Home_Momentum', 'Away_Momentum',
    'Home_HomeFactor', 'Away_AwayFactor',
    'Diff_Form', 'Diff_Attack_Defense', 'Diff_Momentum', # New features
    'B365H', 'B365D', 'B365A'
]

# Ensure numeric types
for col in features:
    df_advanced[col] = pd.to_numeric(df_advanced[col], errors='coerce')

# 2. Train/Test Split
split_idx = int(len(df_advanced) * 0.85)

X = df_advanced[features]
le = LabelEncoder()
y_winner = le.fit_transform(df_advanced['FTR'])
y_goals = (df_advanced['FTHG'] + df_advanced['FTAG']).values

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train_win, y_test_win = y_winner[:split_idx], y_winner[split_idx:]
y_train_goals, y_test_goals = y_goals[:split_idx], y_goals[split_idx:]

# 3. Create Sample Weights (Time Decay)
cutoff_date = pd.Timestamp('2024-08-01')
weights = df_advanced.iloc[:split_idx]['Date'].apply(lambda x: 2.0 if x > cutoff_date else 1.0).values

# 4. Hyperparameter Tuning (Auto-Optimization)
print("🚀 Tuning Model Parameters (this takes ~2 mins)...")
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

xgb_search = XGBClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_search, param_distributions=param_dist,
    n_iter=15, scoring='accuracy', cv=3, verbose=0, random_state=42, n_jobs=-1
)
# Fit search (without weights for stability)
random_search.fit(X_train.values, y_train_win)
best_params = random_search.best_params_
print(f"✅ Best Params: {best_params}")

# 5. Train Final Models
print("Training Final Classifier...")
model_winner = XGBClassifier(**best_params, random_state=42)
model_winner.fit(X_train.values, y_train_win, sample_weight=weights)

print("Training Regressor (Goals)...")
model_goals = XGBRegressor(n_estimators=200, learning_rate=0.03, max_depth=5, random_state=42)
model_goals.fit(X_train.values, y_train_goals, sample_weight=weights)

# 6. Evaluate
acc = model_winner.score(X_test.values, y_test_win)
mae = mean_absolute_error(y_test_goals, model_goals.predict(X_test.values))

print("-" * 30)
print(f"🏆 Final Results:")
print(f"   Winner Accuracy: {acc:.2%}")
print(f"   Goals MAE: {mae:.2f}")
print("-" * 30)

🚀 Tuning Model Parameters (this takes ~2 mins)...
✅ Best Params: {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
Training Final Classifier...
Training Regressor (Goals)...
------------------------------
🏆 Final Results:
   Winner Accuracy: 54.08%
   Goals MAE: 1.31
------------------------------


## 🔮 Real-Time Prediction Engine
This section generates the final report.
It rebuilds the team stats based on the very latest data available (up to today) and feeds it into our trained XGBoost models.

The report highlights:
* **Predicted Winner** (with confidence %).
* **Value Bets:** Where our model sees a higher probability than the bookie's implied odds.
* **Goals Market:** Expected goals and Over/Under recommendations.
* **Handicap:** Estimated score difference.

In [5]:
# Load API Key securely
try:
    from google.colab import userdata
    # Try fetching from Colab Secrets
    API_KEY = userdata.get('ODDS_API_KEY')
except ImportError:
    API_KEY = None
except Exception:
    API_KEY = None

# If not found in Secrets (e.g., someone else running the code), use placeholder
if not API_KEY:
    API_KEY = 'YOUR_API_KEY_HERE'

# Function to fetch fixtures and odds from The Odds API
def get_fixtures_with_odds(api_key):
    print("🌐 Connecting to The Odds API...")

    sport_key = 'soccer_epl'
    url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds/?apiKey={api_key}&regions=uk,eu&markets=h2h&oddsFormat=decimal'

    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ API Error: {response.status_code}")
        # Common check for invalid API key
        if response.status_code == 401:
            print("   👉 Check your API Key!")
        return []

    data = response.json()
    print(f"✅ API returned {len(data)} raw games.")

    fixtures_list = []
    seen_teams = set()

    team_mapping = {
        'Manchester United': 'Man United', 'Manchester City': 'Man City',
        'Tottenham Hotspur': 'Tottenham', 'Newcastle United': 'Newcastle',
        'West Ham United': 'West Ham', 'Wolverhampton Wanderers': 'Wolves',
        'Brighton and Hove Albion': 'Brighton', 'Nottingham Forest': "Nott'm Forest",
        'Sheffield United': 'Sheffield United', 'Luton Town': 'Luton Town',
        'Leeds United': 'Leeds', 'Leicester City': 'Leicester', 'Ipswich Town': 'Ipswich',
        'Brentford': 'Brentford', 'Everton': 'Everton', 'Fulham': 'Fulham',
        'Liverpool': 'Liverpool', 'Arsenal': 'Arsenal', 'Chelsea': 'Chelsea',
        'Aston Villa': 'Aston Villa', 'Crystal Palace': 'Crystal Palace', 'Bournemouth': 'Bournemouth'
    }

    PRIORITY_BOOKIES = ['bet365', 'williamhill', 'unibet', 'pinnacle', 'betfair']
    sorted_data = sorted(data, key=lambda x: x['commence_time'])

    for game in sorted_data:
        home_raw = game['home_team']
        away_raw = game['away_team']

        home_team = team_mapping.get(home_raw, home_raw)
        away_team = team_mapping.get(away_raw, away_raw)

        if home_team in seen_teams or away_team in seen_teams: continue

        b365h, b365d, b365a = 0.0, 0.0, 0.0
        selected_bookie = "None"
        found_bookie = False

        if game['bookmakers']:
            target_bookie = None
            for priority in PRIORITY_BOOKIES:
                target_bookie = next((b for b in game['bookmakers'] if b['key'] == priority), None)
                if target_bookie: break

            if not target_bookie: target_bookie = game['bookmakers'][0]

            selected_bookie = target_bookie['title']
            for market in target_bookie['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        if outcome['name'] == game['home_team']: b365h = outcome['price']
                        elif outcome['name'] == game['away_team']: b365a = outcome['price']
                        elif outcome['name'] == 'Draw': b365d = outcome['price']
                    found_bookie = True

        if found_bookie and b365h > 0:
            fixtures_list.append({
                'Home': home_team, 'Away': away_team,
                'B365H': b365h, 'B365D': b365d, 'B365A': b365a,
                'Bookie': selected_bookie,
                'CommenceTime': game['commence_time']
            })
            seen_teams.add(home_team); seen_teams.add(away_team)

    return fixtures_list[:10]

# Calculate prediction statistics
def get_prediction_inputs(home, away, full_df):
    base = ['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']
    h = full_df[base].copy(); h['Team']=h['HomeTeam']; h['IsHome']=1; h['Pts']=h['FTR'].apply(lambda x: 3 if x=='H' else (1 if x=='D' else 0)); h['GS']=h['FTHG']; h['GC']=h['FTAG']
    a = full_df[base].copy(); a['Team']=a['AwayTeam']; a['IsHome']=0; a['Pts']=a['FTR'].apply(lambda x: 3 if x=='A' else (1 if x=='D' else 0)); a['GS']=a['FTAG']; a['GC']=a['FTHG']
    history = pd.concat([h, a]).sort_values(['Team', 'Date'])

    if home not in history['Team'].unique() or away not in history['Team'].unique(): return None

    h_hist = history[history['Team'] == home].sort_values('Date')
    a_hist = history[history['Team'] == away].sort_values('Date')

    stats = {}
    stats['Home_Form'] = h_hist['Pts'].tail(5).mean()
    stats['Home_Attack'] = h_hist['GS'].tail(5).mean()
    stats['Home_Defense'] = h_hist['GC'].tail(5).mean()
    stats['Home_Momentum'] = h_hist['Pts'].tail(3).sum()
    hh = h_hist[h_hist['IsHome']==1]
    stats['Home_HomeFactor'] = hh['Pts'].tail(5).mean() if not hh.empty else stats['Home_Form']

    stats['Away_Form'] = a_hist['Pts'].tail(5).mean()
    stats['Away_Attack'] = a_hist['GS'].tail(5).mean()
    stats['Away_Defense'] = a_hist['GC'].tail(5).mean()
    stats['Away_Momentum'] = a_hist['Pts'].tail(3).sum()
    aa = a_hist[a_hist['IsHome']==0]
    stats['Away_AwayFactor'] = aa['Pts'].tail(5).mean() if not aa.empty else stats['Away_Form']

    stats['Diff_Form'] = stats['Home_Form'] - stats['Away_Form']
    stats['Diff_Attack_Defense'] = stats['Home_Attack'] - stats['Away_Defense']
    stats['Diff_Momentum'] = stats['Home_Momentum'] - stats['Away_Momentum']
    return stats

# Main Execution

# Check if the user provided an API key in Secrets or directly in the code
if API_KEY == 'YOUR_API_KEY_HERE' or API_KEY is None:
    print("⚠️ SECURITY ALERT: No API Key found in Secrets!")
    print("   Please add 'ODDS_API_KEY' to Colab Secrets (Key icon on the left).")
    next_fixtures = []
else:
    next_fixtures = get_fixtures_with_odds(API_KEY)

if not next_fixtures:
    print("⚠️ No games found. Using fallback.")
    next_fixtures = [{'Home': 'Arsenal', 'Away': 'Liverpool', 'B365H': 2.40, 'B365D': 3.40, 'B365A': 2.90, 'Bookie': 'Manual'}]

print("\n" + "="*75)
print("🤖 FINAL AI REPORT (SECURE MODE)")
print("Model Accuracy: ~54% | Data Source: The Odds API")
print("="*75)

for fixture in next_fixtures:
    stats = get_prediction_inputs(fixture['Home'], fixture['Away'], df)

    if stats:
        stats['B365H'] = fixture['B365H']
        stats['B365D'] = fixture['B365D']
        stats['B365A'] = fixture['B365A']

        row = pd.DataFrame([stats])
        row = row[features]

        probs = model_winner.predict_proba(row.values)[0]
        pred_class = model_winner.predict(row.values)[0]
        pred_goals = model_goals.predict(row.values)[0]

        if pred_class == 2: winner = fixture['Home']
        elif pred_class == 0: winner = fixture['Away']
        else: winner = "DRAW"

        conf = max(probs)

        print(f"\n⚽ {fixture['Home']} vs {fixture['Away']}")
        print(f"   📊 Odds ({fixture.get('Bookie', 'Unknown')}): {fixture['B365H']} | {fixture['B365D']} | {fixture['B365A']}")
        print(f"   🏆 Prediction: {winner} (Conf: {conf:.1%})")

        if probs[2] > (1/fixture['B365H']) + 0.05: print(f"   💰 VALUE HOME (Model: {probs[2]:.0%} vs Implied: {1/fixture['B365H']:.0%})")
        elif probs[0] > (1/fixture['B365A']) + 0.05: print(f"   💰 VALUE AWAY (Model: {probs[0]:.0%} vs Implied: {1/fixture['B365A']:.0%})")

        print(f"   🥅 Exp. Goals: {pred_goals:.2f}")
        if pred_goals > 2.7: print("   📈 Pick: OVER 2.5 Goals")

    else:
        print(f"⚠️ Missing historical data for {fixture['Home']} vs {fixture['Away']}")

🌐 Connecting to The Odds API...
✅ API returned 19 raw games.

🤖 FINAL AI REPORT (SECURE MODE)
Model Accuracy: ~54% | Data Source: The Odds API

⚽ Crystal Palace vs Aston Villa
   📊 Odds (William Hill): 3.0 | 3.25 | 2.25
   🏆 Prediction: Aston Villa (Conf: 40.3%)
   🥅 Exp. Goals: 2.08

⚽ Bournemouth vs Tottenham
   📊 Odds (William Hill): 2.1 | 3.4 | 3.1
   🏆 Prediction: Bournemouth (Conf: 40.5%)
   🥅 Exp. Goals: 2.05

⚽ Brentford vs Sunderland
   📊 Odds (William Hill): 1.8 | 3.5 | 4.2
   🏆 Prediction: Brentford (Conf: 44.0%)
   🥅 Exp. Goals: 2.33

⚽ Man City vs Brighton
   📊 Odds (William Hill): 1.4 | 4.8 | 6.5
   🏆 Prediction: Man City (Conf: 51.2%)
   🥅 Exp. Goals: 3.15
   📈 Pick: OVER 2.5 Goals

⚽ Fulham vs Chelsea
   📊 Odds (William Hill): 3.3 | 3.4 | 2.1
   🏆 Prediction: Chelsea (Conf: 41.2%)
   🥅 Exp. Goals: 2.61

⚽ Everton vs Wolves
   📊 Odds (William Hill): 1.75 | 3.4 | 4.75
   🏆 Prediction: Everton (Conf: 51.0%)
   💰 VALUE AWAY (Model: 27% vs Implied: 21%)
   🥅 Exp. Goals: 3.51